In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8540,2024-05-16,Eua Nba,21:30,Minnesota Timberwolves,Denver Nuggets,1.78,2.11,202.5,1.81,2.04,-2.5,2.01,2.45,UDcZnEI6,0.561798,0.473934,0.552486,0.490196,0.035731,0.8,1.0,NaN,NaN,232.364,77.361160,0.332931,198.386,78.710153,0.396753,180.100,225.542,179.76,261.05,0.0,0.0,0.0,0.0,0.119972,0.084485,0.139519,-1.53,-0.306,-2.549020,0.588712,0.4,-0.188712,3.30,0.660,1.681818,0.541344,0.6,0.058656
8541,2024-05-16,Itália Liga A,15:00,Reggiana,Venezia,1.72,2.14,156.5,1.84,1.92,-3.5,2.03,2.60,nRoreG6r,0.581395,0.467290,0.543478,0.520833,0.048685,0.0,0.8,NaN,NaN,167.986,77.730675,0.462721,160.576,24.775837,0.154294,152.944,141.068,184.80,132.00,0.0,0.0,0.0,0.0,0.153878,0.030090,0.174104,3.11,0.622,1.157556,0.000000,0.0,0.000000,-1.50,-0.300,-3.800000,0.567326,0.5,-0.067326
8542,2024-05-16,Itália Liga A,15:45,Tortona,Virtus Bologna,2.71,1.47,161.5,1.89,1.90,3.5,2.04,1.27,G6waiY57,0.369004,0.680272,0.529101,0.526316,0.049276,0.0,0.0,NaN,NaN,142.018,22.765610,0.160301,132.744,21.742573,0.163793,129.744,125.320,109.81,147.74,0.0,0.0,0.0,0.0,0.419527,0.003731,0.328986,1.60,0.320,5.343750,0.000000,0.0,0.000000,1.00,0.200,2.350000,0.703062,0.6,-0.103062
8543,2024-05-16,Argentina Liga A,20:30,Gimnasia,Boca Juniors,1.75,1.98,158.5,1.80,1.86,-2.5,2.02,2.38,K8ep8oC3,0.571429,0.505051,0.555556,0.537634,0.076479,0.6,0.4,NaN,NaN,127.090,18.045434,0.141989,110.704,10.187457,0.092024,125.546,122.964,113.92,117.30,0.0,0.0,0.0,0.0,0.087204,0.023184,0.115708,2.18,0.436,1.720183,0.000000,0.0,0.000000,-0.48,-0.096,-10.208333,0.000000,0.0,0.000000
8544,2024-05-16,Argentina Liga A,21:30,La Union,San Lorenzo,1.71,2.03,153.5,1.81,1.85,-2.5,1.97,2.45,rZGmQkc8,0.584795,0.492611,0.552486,0.540541,0.077406,0.6,0.4,NaN,NaN,237.940,85.220523,0.358160,195.218,98.857242,0.506394,193.590,219.842,141.10,103.95,0.0,0.0,0.0,0.0,0.121002,0.015456,0.153580,-1.85,-0.370,-1.918919,0.000000,0.0,0.000000,-1.02,-0.204,-5.049020,0.000000,0.0,0.000000
8545,2024-05-16,Libano Divisão 1,13:00,Beirut Club,Homenetmen Beirut,1.70,2.05,179.5,1.85,1.85,-2.5,1.94,2.50,6R05axfM,0.588235,0.487805,0.540541,0.540541,0.076040,1.0,0.4,NaN,NaN,148.726,39.170905,0.263376,126.908,36.696671,0.289160,131.604,205.534,178.20,190.65,0.0,0.0,0.0,0.0,0.131993,0.000000,0.178369,-1.80,-0.360,-1.944444,0.000000,0.0,0.000000,6.10,1.220,0.860656,0.000000,0.0,0.000000
8546,2024-05-16,México Cibacopa,00:15,Caballeros de Culiacan,Rayos de Hermosillo,2.20,1.62,176.5,1.85,1.85,2.5,1.90,1.39,069xSuC6,0.454545,0.617284,0.540541,0.540541,0.071829,0.2,1.0,NaN,NaN,179.162,17.543174,0.097918,177.786,23.798115,0.133858,179.162,177.786,154.77,162.84,0.0,0.0,0.0,0.0,0.214724,0.000000,0.219225,-0.80,-0.160,-7.500000,0.000000,0.0,0.000000,2.36,0.472,1.313559,0.000000,0.0,0.000000
8547,2024-05-16,México Cibacopa,00:15,Halcones de Obregon,Angeles CDMX,1.67,2.12,181.5,1.85,1.85,-3.5,2.03,2.91,rVbsRaRC,0.598802,0.471698,0.540541,0.540541,0.070501,0.8,0.4,NaN,NaN,171.332,27.842167,0.162504,170.978,27.728296,0.162175,187.654,158.460,162.24,193.44,0.0,0.0,0.0,0.0,0.167915,0.000000,0.251925,2.94,0.588,1.139456,0.000000,0.0,0.000000,1.68,0.336,3.333333,0.000000,0.0,0.000000
8548,2024-05-16,Rússia Liga Vtb United,13:30,Zenit Petersburg,CSKA Moscow,1.69,2.19,161.5,1.80,1.94,-3.5,2.00,2.80,tIx37p15,0.591716,0.456621,0.555556,0.515464,0.048337,0.6,0.8,NaN,NaN,165.026,26.988248,0.163539,167.380,11.984749,0.071602,145.464,160.052,170.19,160.38,0.0,0.0,0.0,0.0,0.182244,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Nba,Minnesota Timberwolves,Denver Nuggets,202.5,1.81,1.0,Over
1,15:00,Itália Liga A,Reggiana,Venezia,156.5,1.84,1.0,Over
2,20:30,Argentina Liga A,Gimnasia,Boca Juniors,158.5,1.80,1.0,Over
3,21:30,Argentina Liga A,La Union,San Lorenzo,153.5,1.81,1.0,Over
4,13:00,Libano Divisão 1,Beirut Club,Homenetmen Beirut,179.5,1.85,1.0,Over
5,00:15,México Cibacopa,Halcones de Obregon,Angeles CDMX,181.5,1.85,1.0,Over
6,13:30,Rússia Liga Vtb United,Zenit Petersburg,CSKA Moscow,161.5,1.80,1.0,Over
